In [1]:
import os
import keras
import numpy as np
import sklearn.model_selection as ms
import sklearn.decomposition as decomp
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import librosa
import librosa.display as display

Using TensorFlow backend.


In [ ]:
#load in the saved data
X = np.load('data_mat/X3.npy')
Y = np.load('data_mat/Y3.npy')

In [31]:
X_test = np.load('mfcc_vals/X_test.npy')
X_train = np.load('mfcc_vals/X_train.npy')
Y_test = np.loadtxt('mfcc_vals/Y_test.txt')
Y_train = np.loadtxt('mfcc_vals/Y_train.txt')
Y_train.shape

(400, 5)

In [27]:
def model(input_shape):
    X_inputs = keras.layers.Input(input_shape)
    X = X_inputs
    
    X = keras.layers.Conv1D(filters=16, kernel_size=64, dilation_rate=8,
                            padding='same', data_format="channels_last")(X)
    X = keras.layers.Dropout(0.5)(X)    
    X = keras.layers.AveragePooling1D(pool_size=32)(X)
    X = keras.layers.BatchNormalization(axis = 2)(X)
    X = keras.layers.Activation('relu')(X)
    
    X = keras.layers.Conv1D(filters=64, kernel_size=16, dilation_rate=2, padding='valid', data_format="channels_last")(X)
    X = keras.layers.Dropout(0.5)(X)    
    X = keras.layers.AveragePooling1D(pool_size=4)(X)
    X = keras.layers.BatchNormalization(axis = 2)(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Flatten()(X)
#     X = keras.layers.Dropout(0.5)(X)    
    X = keras.layers.Dense(5, activation='softmax')(X)
    
    model = keras.models.Model(inputs=X_inputs, outputs=X, name='cnn')
    return model

In [28]:
m = model(X_train.shape[1:])
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
m.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [29]:
m.summary()

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 1290, 20)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1290, 16)          20496     
_________________________________________________________________
dropout_8 (Dropout)          (None, 1290, 16)          0         
_________________________________________________________________
average_pooling1d_7 (Average (None, 40, 16)            0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 40, 16)            64        
_________________________________________________________________
activation_7 (Activation)    (None, 40, 16)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 10, 64)            16448   

In [30]:
m.fit(X_train, Y_train, epochs=50, batch_size=64)

Epoch 1/50
400/400 [==============================] - 2s 4ms/step - loss: 1.1422 - categorical_accuracy: 0.5675
Epoch 2/50
400/400 [==============================] - 1s 2ms/step - loss: 0.6955 - categorical_accuracy: 0.7225
Epoch 3/50
400/400 [==============================] - 1s 2ms/step - loss: 0.5835 - categorical_accuracy: 0.7875
Epoch 4/50
400/400 [==============================] - 1s 2ms/step - loss: 0.4766 - categorical_accuracy: 0.8500
Epoch 5/50
400/400 [==============================] - 1s 2ms/step - loss: 0.4639 - categorical_accuracy: 0.8125
Epoch 6/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3989 - categorical_accuracy: 0.8750
Epoch 7/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3706 - categorical_accuracy: 0.8750
Epoch 8/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3763 - categorical_accuracy: 0.8525
Epoch 9/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3442 - categorical_accuracy:

In [32]:
preds = m.evaluate(X_train, Y_train)
print ("Loss = " + str(preds[0]))
print ("Train Accuracy = " + str(preds[1]))

400/400 [==============================] - 1s 1ms/step
Loss = 0.10349284708499909
Train Accuracy = 0.9775000214576721


In [33]:
preds = m.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

100/100 [==============================] - 0s 1ms/step
Loss = 0.44623344898223877
Test Accuracy = 0.8500000238418579


In [ ]:
def extract_all_features(X, model, layer_nums, batch_num):
    """Extract all the features by taking out the activation output of the layers specified in layer_nums
    
        X : the data that we are trying to extract features from
        model: the model that we are using to extract the features
        layer_nums: the layer number that we want to use as feature extractors
    """
    outputs = [model.layers[layer].output for layer in layer_nums]
    f = keras.backend.function([model.input, keras.backend.learning_phase()], outputs)
    ## split up into smaller chunks
    layer_outs = []
    m = X.shape[0]
    for i in range(batch_num):
        layer_outs += [f([X[(m // batch_num * i):(m // batch_num * (i+1)), :, :], 0.])]
    result = []
    for i in range(len(layer_nums)):
        layer_outs2 = []
        for batch in range(batch_num):
            activations = layer_outs[batch][i]
            activations = activations.reshape(activations.shape[0], -1)
            layer_outs2 += [activations]
        result += [np.vstack(np.array(layer_outs2))]
    return result